In [94]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.cluster import KMeans 
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.preprocessing import scale
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, LatentDirichletAllocation
import pandas as pd


In [95]:
data = fetch_20newsgroups()
print(data.target_names)
x, y = fetch_20newsgroups(return_X_y=True)

#fixing dimensions (not working)
# x = np.array(x)
# y= np.array(y)
# x = x[:,np.newaxis]
# y = y[:,np.newaxis]

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [96]:
np.unique(y, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19]),
 array([480, 584, 591, 590, 578, 593, 585, 594, 598, 597, 600, 595, 591,
        594, 593, 599, 546, 564, 465, 377], dtype=int64))

In [97]:
cluster_number = len(np.unique(y))
cluster_number

20

In [98]:
# shamelessly stealing Adrien's nice function for clustering benchmark:, 

def bench_clust(estimator, data, ground_truth = None,):
    estimator.fit(data)
    scores = [
        "adjusted_rand_score",
        "adjusted_mutual_info_score",
        "silhouette_score",
        "homogeneity_score",
        "completeness_score",
        "v_measure_score",
    ]
    results = []
    for s in scores:
        if s not in ["silhouette_score"]:
            #where ground truth is provided
            score = getattr(metrics, s)(ground_truth, estimator.labels_)
        elif ground_truth is not None:
            score = getattr(metrics, s)(data, estimator.labels_)
        else: 
            continue

        results.append(score)
    return pd.DataFrame([results], columns=scores, index=['Results'])

In [100]:
#building train test split : 
x_train, x_test, y_train, y_test = train_test_split(x,y, random_state=42)


# failed to build a pipeline, it seems to cause a data dimension error that does not occur while working "the handy way", 
# # using CountVectorizer to extract tokens matrix, and kmeans as a model, in a sequence.
vect = CountVectorizer()
x_train_vec = vect.fit_transform(x_train)

model = KMeans(n_clusters= cluster_number)

#let's give it a try : 
bench_clust(model, x_train_vec, y_train)

c:\Users\Thaz\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


,adjusted_rand_score,adjusted_mutual_info_score,silhouette_score,homogeneity_score,completeness_score,v_measure_score
Results,0.009789,0.036738,0.114842,0.030586,0.074524,0.043372


In [102]:
#test with tfidf

better_vec = TfidfVectorizer()
x_train_vec2 = better_vec.fit_transform(x_train)

#let's give it a try : 
bench_clust(model, x_train_vec2, y_train)

c:\Users\Thaz\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


,adjusted_rand_score,adjusted_mutual_info_score,silhouette_score,homogeneity_score,completeness_score,v_measure_score
Results,0.079343,0.247133,-0.006622,0.230287,0.281367,0.253277
